### 1.Load and normalizing the CIFAR10 datasets

In [81]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 2. Define a Convolutional Neural Network

In [82]:
import torch.nn as nn
import torch.nn.functional as F


class Net1(nn.Module): #[Conv+MP]*2+[FC]*3
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 10)
        self.comment = '{Net1||[Conv+MP]*2+[FC]*3}'

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Net2(nn.Module): #[Conv+MP]*3+[FC]*3
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 10)
        self.comment = '{Net2||[Conv+MP]*3+[FC]*3}'

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 2 * 2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Net3(nn.Module): #[Conv+MP]*2+[FC]*4
    def __init__(self):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        self.comment = '{Net3||[Conv+MP]*2+[FC]*4}'

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class Net4(nn.Module): #[Conv+MP]*3+[FC]*4
    def __init__(self):
        super(Net4, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        self.comment = '{Net4||[Conv+MP]*3+[FC]*4}'

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 2 * 2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


### 3. Define a Loss function

In [83]:
import torch.optim as optim

loss_func = nn.CrossEntropyLoss()

### 4. Define a Training and Testing function

In [84]:
from tensorboardX import SummaryWriter


def Training(net, batch_size, epochs, learning_rate):
    
    writer = SummaryWriter(
    comment='{} batch_size={}, epochs={}, lr={}'.format(net.comment, batch_size, epochs, learning_rate)
    )
    
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):

        running_loss = 0.0
        total_batch = len(trainset)//batch_size
    
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
    
            outputs = net(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
        
            _, predicted = torch.max(outputs, 1)
            correct = (predicted == labels).sum().item()
            total = labels.size(0)
        
        
            #Recoding on tensorboard
            writer.add_scalar('Train/Loss', loss, epoch*total_batch+i)
            writer.add_scalar('Train/Accuracy', correct/total, epoch*total_batch+i)

            """
            running_loss += loss.item()
            printing_unit = total_batch // 5
            if i % printing_unit == printing_unit-1 :    # print loss for 5 times per epoch
                print('[{}, {}] loss: {:f}'.format(epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
            """

    print('Finished Training on' + net.comment)
    
    return net, writer
    

def Testing(net, batch_size, writer):
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network{} on the 10000 test images: {}'.format(net.comment, correct/total))

    writer.add_scalar('Test/Accuracy', correct/total, 1)
    
    writer.close()
    
    return correct/total

### 5. Training and Testing networks

In [85]:
Nets = [Net1(), Net2(), Net3(), Net4()]
Batch_sizes = [4,10,25,100]
epochs = 10
Learning_rates = [0.0008, 0.001, 0.0012]

accuracy = []
i = 0

for net in Nets:
    accuracy.append([])
    print(net)
    j = 0
    for batch_size in Batch_sizes:
        accuracy[i].append([])
        for learning_rate in Learning_rates:
            net, writer = Training(net, batch_size, epochs, learning_rate)
            accuracy[i][j].append(Testing(net, batch_size, writer))
        j += 1
    i += 1

Net1(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
Finished Training on{Net1||[Conv+MP]*2+[FC]*3}
Accuracy of the network{Net1||[Conv+MP]*2+[FC]*3} on the 10000 test images: 0.6772
Finished Training on{Net1||[Conv+MP]*2+[FC]*3}
Accuracy of the network{Net1||[Conv+MP]*2+[FC]*3} on the 10000 test images: 0.6668
Finished Training on{Net1||[Conv+MP]*2+[FC]*3}
Accuracy of the network{Net1||[Conv+MP]*2+[FC]*3} on the 10000 test images: 0.6732
Finished Training on{Net1||[Conv+MP]*2+[FC]*3}
Accuracy of the network{Net1||[Conv+MP]*2+[FC]*3} on the 10000 test images: 0.6886
Finished Training on{Net1||[Conv+MP]*2+[FC]*3}
Accuracy of the network{Net1||[Conv

### 6. Finding Maximum

In [87]:
print(accuracy)

[[[0.6772, 0.6668, 0.6732], [0.6886, 0.6881, 0.6822], [0.6843, 0.6794, 0.666], [0.6886, 0.684, 0.6785]], [[0.7102, 0.6929, 0.6876], [0.7386, 0.7132, 0.6876], [0.7271, 0.7367, 0.7336], [0.7387, 0.7393, 0.7345]], [[0.7029, 0.6923, 0.6513], [0.7082, 0.6629, 0.7053], [0.7107, 0.6964, 0.7008], [0.711, 0.702, 0.7027]], [[0.6927, 0.7166, 0.6739], [0.7406, 0.739, 0.7242], [0.743, 0.7438, 0.7424], [0.7491, 0.7442, 0.7478]]]


In [93]:
acc_max = 0.0
location = [0,0,0]
for i in range(len(Nets)):
    for j in range(len(Batch_sizes)):
        for k in range(len(Learning_rates)):
            if acc_max < accuracy[i][j][k]:
                acc_max = accuracy[i][j][k]
                location = [i,j,k]
print('Maximum accuracy is {} on Net{} with batch size of {} and {} learning rate'
      .format(acc_max, location[0]+1, Batch_sizes[location[1]], Learning_rates[location[2]]))

Maximum accuracy is 0.7491 on Net4 with batch size of 100 and 0.0008 learning rate
